We associate to each person a life-style vector (features : how much he likes politics,...)
We propose a friend-matching matrix to represent the similarity between persons' life styles

In [1]:
import scipy.sparse as sparse
import random 
import pandas as pd

import numpy as np
import scipy 


In [2]:
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.layers import Dot
from tensorflow.keras.models import Model

In [3]:
class Env:

    def __init__(self, users_feat, seed=None):
        self.users_feat = users_feat
        nb_users , k =users_feat.shape
        self.k=k
        self.nb_users=nb_users
        self.reward_matrix =f(self.users_feat,self.nb_users) 
        #self.possible_match =np.zeros((self.reward_matrix.shape[0],self.reward_matrix.shape[0]))
        self.possible_match =np.identity((self.reward_matrix.shape[0]))
        self.interactions =  sparse.random(nb_users, nb_users, density=0.25)

        self.nb_users = nb_users
    def step(self):
        """ Play an action """
        #choose a random user
        currUserId =np.random.randint(low=0, high=self.nb_users)
        #list of possible matches
        list_possible_friends= np.where(self.possible_match[currUserId,:]==0)[0]
        
        return currUserId, list_possible_friends
    
    def update(self, user, friend, reward):
        self.possible_match[user, friend] =1
        self.interactions = sparse.csr_matrix(self.interactions)
        self.interactions[user, friend]=reward
        #self.interactions.toarray()
        return self.interactions
     
    def reset(self, seed=None):
        np.random.seed(42)
        self.interactions = sparse.random(nb_users, nb_users, density=0.25)
        self.interactions.setdiag(np.ones(nb_users))
        self.interactions.toarray()
        rows = sparse.find(self.interactions)[0]
        cols = sparse.find(self.interactions)[1]
        possible_match =np.identity((self.reward_matrix.shape[0]))
        for i in range(len(rows)):
            possible_match[rows[i],cols[i]]=1
        self.possible_match = possible_match
        return self.possible_match, self.interactions
    
    def calc_real_reward(self, pred_reward ):
        return (0.6*pred_reward + 0.4*np.random.uniform(0, 1))

In [4]:
def f(users,n):
  r= np.zeros((n,n))
  for user, u in enumerate(users):
    for user2, u2  in enumerate (users):
      u_norm=np.linalg.norm(u)
      u2_norm=np.linalg.norm(u2)
      reward = round(np.dot(u,u2)/(u_norm*u2_norm)*5)+1
      r[user,user2]=reward
      #r[user,item]=round(users[i]*items[j]/ (np.linalg.norm(users)*np.linalg.norm(items))*5)+1

  return r

In [5]:
class RandomAgent:
  def __init__(self):
    pass
  def action(self, user_id, items_available):
    return np.random.choice(items_available)

In [6]:
nb_users = 10
means= np.array([0,0,0])
stds = np.array([1.,3.,5.])
k=3
users_features = np.random.normal(loc=means, scale=stds, size=(nb_users,k))


In [7]:
#env = Env(users_features)
#agent=RandomAgent()
#nb_iters = 20

#for t in range(nb_iters):
#    user_id, friends_to_recommend= env.step()
#    friend_chosen=agent.action(user_id, friends_to_recommend)
#    print("user_id =",user_id, "items_to_recommend=",friends_to_recommend,"friend_chosen=",friend_chosen)
#    reward = env.update(user_id,friend_chosen)
#    print("reward:{}\n".format(reward))

In [8]:
def similarity(interactions):
    
    # vecteur contenant pour chaque utilisateur le nombre de notes données
    r_user = (interactions>0).sum(axis=1)  
    
    # vecteur contenant pour chaque utilisateur la moyenne des notes données
    m_user = np.divide(interactions.sum(axis=1) , r_user, where=r_user!=0)
    
    # Notes recentrées par la moyenne par utilisateur : chaque ligne i contient le vecteur \bar r_i
    interactions_ctr = interactions.T - ((interactions.T!=0) * m_user)
    interactions_ctr = interactions_ctr.T

    # Matrice de Gram, contenant les produits scalaires
    sim = interactions_ctr.dot(interactions_ctr.T)
    
    # Renormalisation
    norms = np.array([np.sqrt(np.diagonal(sim))])
    norms[0][7]=1
    
    sim = sim / norms / norms.T  
    # (En numpy, diviser une matrice par un vecteur ligne (resp. colonne) 
    # revient à diviser chaque ligne (resp. colonne) terme à terme par les éléments du vecteur)
    
    return sim

In [9]:
def phi(x):
    return np.maximum(x,0)

In [10]:
def predict_interactions(interactions,sim,phi=(lambda x:x)):
    
    wsum_sim = np.abs(phi(sim)).dot(interactions>0)
    return np.divide(phi(sim).dot(interactions) , wsum_sim, where= wsum_sim!=0)

In [11]:
class MemoryBasedAgent:
    def __init__(self, interactions):
        self._interactions=interactions
        pass
    def action(self, user_id, ratings, friends_to_recommend):
        #print('ratings avant', ratings)
        ratings = ratings.toarray()
        #print('ratings apres',(ratings))
        sim = similarity(ratings)
        #print('similarity', sim)
        pred_interactions = predict_interactions(ratings,sim,phi)
        #print('pred_interactions', pred_interactions)
        ratings_of_user = pred_interactions[user_id][:]
        idx_max = friends_to_recommend[0]
        max_friend = ratings_of_user[friends_to_recommend[0]]
        #print('ratings_user', ratings_of_user)
        for j in friends_to_recommend:
            if (ratings_of_user[j]>max_friend):
                max_friend = ratings_of_user[j]
                idx_max = j
        pred_reward = ratings_of_user[idx_max]
        
        return idx_max, pred_reward

In [12]:
env = Env(users_features)
possible_match, interactions = env.reset(seed=None)
agent=MemoryBasedAgent(interactions)
nb_iters = 20

for t in range(nb_iters):
    user_id, friends_to_recommend= env.step()
    friend_chosen, pred_reward =agent.action(user_id, interactions, friends_to_recommend)
    print("user_id =",user_id, "items_to_recommend=",friends_to_recommend,"friend_chosen=",friend_chosen)
    real_reward = env.calc_real_reward(pred_reward)
    #print('int _avant',interactions.toarray())
    interactions = env.update(user_id,friend_chosen,real_reward)
    print('real_reward', real_reward)
    #print('int',interactions.toarray())
    #print("reward:{}\n".format(reward))

user_id = 2 items_to_recommend= [0 3 5 6 7 8 9] friend_chosen= 0
real_reward 0.6125716742746937
user_id = 6 items_to_recommend= [0 1 3 4 5 8 9] friend_chosen= 0
real_reward 0.1901480892728447
user_id = 3 items_to_recommend= [0 1 2 4 6 9] friend_chosen= 9
real_reward 0.9630265895704372
user_id = 6 items_to_recommend= [1 3 4 5 8 9] friend_chosen= 3
real_reward 0.8417669517111269
user_id = 3 items_to_recommend= [0 1 2 4 6] friend_chosen= 4
real_reward 0.7407174130917993
user_id = 6 items_to_recommend= [1 4 5 8 9] friend_chosen= 9
real_reward 0.6937165349077696
user_id = 5 items_to_recommend= [0 1 2 3 6 7 8 9] friend_chosen= 2
real_reward 0.9521871356061031
user_id = 9 items_to_recommend= [0 1 2 4 5 7 8] friend_chosen= 4
real_reward 0.7676785996808464
user_id = 8 items_to_recommend= [0 2 3 4 5 6 7 9] friend_chosen= 0
real_reward 0.6421977039321082
user_id = 3 items_to_recommend= [0 1 2 6] friend_chosen= 6
real_reward 0.698623782203639
user_id = 9 items_to_recommend= [0 1 2 5 7 8] friend_ch

C:\Users\user\Anaconda3\lib\site-packages\scipy\sparse\_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [13]:
env = Env(users_features)
possible_match, interactions = env.reset(seed=None)

In [14]:
print(type(interactions.todense()))

<class 'numpy.matrix'>


In [15]:
interactions2 = interactions.todense()

In [16]:
def conv_to_df(interactions):
    nb_users = len(interactions)
    M=np.zeros((nb_users*nb_users,3))
    for i in range (nb_users):
        for j in range(nb_users):
            M[j+i*nb_users][0]=i
            M[j+i*nb_users][1]=j
            M[j+i*nb_users][2]=interactions2[i,j]
    df = pd.DataFrame(M,columns=['user_id','user_id','interactions'])
    return df

In [17]:
conv_to_df(interactions2)

,user_id,user_id,interactions
0,0.0,0.0,1.000000
1,0.0,1.0,0.449451
2,0.0,2.0,0.000000
3,0.0,3.0,0.668841
4,0.0,4.0,0.000000
...,...,...,...
95,9.0,5.0,0.000000
96,9.0,6.0,0.110891
97,9.0,7.0,0.000000
98,9.0,8.0,0.000000


In [18]:
class RegressionModel(Model):
    def __init__(self, embedding_size, max_user_id):
        super().__init__()
        
        self.user_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_user_id + 1,
                                        input_length=1,
                                        name='user_embedding')
        self.item_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_item_id + 1,
                                        input_length=1,
                                        name='item_embedding')
        
        # The following two layers don't have parameters.
        self.flatten = Flatten()
        self.dot = Dot(axes=1)
        
    def call(self, inputs):
        user_inputs = inputs[0]
        item_inputs = inputs[1]
        
        user_vecs = self.flatten(self.user_embedding(user_inputs))
        item_vecs = self.flatten(self.item_embedding(item_inputs))
        
        y = self.dot([user_vecs, item_vecs])
        return y


model = RegressionModel(64, nb_users)
model.compile(optimizer="adam", loss='mae')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


NameError: name 'max_item_id' is not defined